In [5]:
function repeatlength(A, i)
    k = 1
    @inbounds while i + k ≤ lastindex(A) && isequal(A[i+k], A[i])
        k += 1
    end
    k
end

"""
    RepVal(A)
Iterator generating the all (repeat length, value)-tuples in `A`.
"""
struct RepVal{T} A::T end
Base.IteratorSize(::Type{RepVal{T}}) where T = Base.SizeUnknown()
Base.eltype(x::RepVal) = Tuple{Int, eltype(x.A)} # (rep.len., value)

Base.iterate(x::RepVal) = iterate(x, firstindex(x.A))
function Base.iterate(x::RepVal, i::Int)
    i > lastindex(x.A) && return nothing
    k = repeatlength(x.A, i)
    (k, x.A[i]), i + k
end

maxrep_maxval(A) = maximum(RepVal(A))
negrep((k, v)) = (-k, v)
maxrep_minval(A) = negrep(minimum(negrep, RepVal(A)))

@doc RepVal

```
RepVal(A)
```

Iterator generating the all (repeat length, value)-tuples in `A`.


In [7]:
A = [1, 2, 3, 3, 3, 1, 1, 1, NaN, NaN, NaN, 2, 2, 3]
@show A
RepVal(A) |> collect

A = [1.0, 2.0, 3.0, 3.0, 3.0, 1.0, 1.0, 1.0, NaN, NaN, NaN, 2.0, 2.0, 3.0]


7-element Vector{Tuple{Int64, Float64}}:
 (1, 1.0)
 (1, 2.0)
 (3, 3.0)
 (3, 1.0)
 (3, NaN)
 (2, 2.0)
 (1, 3.0)

In [3]:
maxrep_maxval(A)

(3, NaN)

In [4]:
maxrep_minval(A)

(3, 1.0)

In [5]:
@code_warntype iterate(RepVal(A), 1)

Variables
  #self#::Core.Const(iterate)
  x::RepVal{Vector{Float64}}
  i::Int64
  k::Int64

Body::Union{Nothing, Tuple{Tuple{Int64, Float64}, Int64}}
1 ─       Core.NewvarNode(:(k))
│   %2  = Base.getproperty(x, :A)::Vector{Float64}
│   %3  = Main.lastindex(%2)::Int64
│   %4  = (i > %3)::Bool
└──       goto #3 if not %4
2 ─       return Main.nothing
3 ─ %7  = Base.getproperty(x, :A)::Vector{Float64}
│         (k = Main.repeatlength(%7, i))
│   %9  = k::Int64
│   %10 = Base.getproperty(x, :A)::Vector{Float64}
│   %11 = Base.getindex(%10, i)::Float64
│   %12 = Core.tuple(%9, %11)::Tuple{Int64, Float64}
│   %13 = (i + k)::Int64
│   %14 = Core.tuple(%12, %13)::Tuple{Tuple{Int64, Float64}, Int64}
└──       return %14


https://discourse.julialang.org/t/minimum-mode-problem-minimum-number-maximum-repetition/66749/2

In [6]:
function findn(x,y=zeros(Int,maximum(x)))
    m = 0
    fill!(y,0)
    @inbounds for i in x
        y[i] += 1
        m = max(y[i],m)
    end
    return findfirst(isequal(m),y)
end

findn (generic function with 2 methods)

In [7]:
using StatsBase, DataStructures

function findn_jling(x)
    cm = sort!(OrderedDict(countmap(x)); byvalue = true)
    last(cm.keys), last(cm.vals)
end

findn_jling (generic function with 1 method)

In [8]:
function minMode(c)   
    minVal = -1
    i = 0
    
    i = i+1
    for row in eachrow(c)
    row = collect(row)
    temp = filter(x -> x!=0, row)
    count = counter(temp)
    sortedCollection = sort(collect(count), by=x->x[2], rev=true)
    minVal = sortedCollection[1][1]
    repetitions = sortedCollection[1][2]
    for (key, value) in sortedCollection
        if(value == repetitions)
            if(key < minVal)
                minVal = key
            end
        end 
        end
    end
    
    return minVal
end

minMode (generic function with 1 method)

In [13]:
using StatsBase
swap_negval((val, rep)) = (rep, -val)
inv_swap_negval((rep, val)) = (-val, rep)
minmode(X) = inv_swap_negval(maximum(swap_negval, countmap(X)))k

minmode (generic function with 1 method)

In [26]:
X = rand(1:10000:10^8, 10^6)
X'

1×1000000 adjoint(::Vector{Int64}) with eltype Int64:
 4170001  46020001  82500001  61360001  …  8940001  83130001  75350001

In [3]:
using BenchmarkTools

In [17]:
@btime maxrep_minval($X)

  132.700 μs (0 allocations: 0 bytes)


(2, 2201)

In [27]:
@btime findn($X)

  202.202 ms (2 allocations: 762.86 MiB)


8240001

In [19]:
@btime minMode($X)

  35.930 ms (1100000 allocations: 111.39 MiB)


93701

In [28]:
@btime minmode($X)

  13.802 ms (36 allocations: 15.73 MiB)


(8240001, 144)

In [29]:
@btime findn_jling($X)

  14.312 ms (82 allocations: 16.61 MiB)


(8240001, 144)